In [1]:
import numpy as np

In [36]:
edges1 = {}

for i in range(50):
    cluster_i = i // 25
    N_in = np.random.binomial(25, 0.6)
#     N_in = np.random.binomial(100, 0.5)
    in_poi = list(cluster_i * 25 + np.random.choice(25, N_in))
    N_out = np.random.binomial(25, 0.1)
    out_poi = list(np.random.choice(25, N_out))
    for j in in_poi + out_poi:
        if j not in edges1:
            edges1[j] = {}
        edges1[j][i] = (np.random.poisson(1) + 1) 
        

In [38]:
redges1 = {}

for j in edges1:
    for i in edges1[j]:
        if i not in redges1:
            redges1[i] = {}
        redges1[i][j] = edges1[j][i]

In [48]:
degree

array([ 1,  1, 14,  1, 33,  1, 19,  1,  1, 18,  4,  3,  1, 20,  1, 12,  1,
        1,  1, 28, 19, 12,  1,  1,  3,  1,  1,  1,  1,  1, 15,  1,  1,  2,
        3,  1,  1,  1,  1,  1, 21,  4,  2, 44,  1,  3,  1, 19,  1,  8,  1,
        7,  1,  1,  1, 43, 11,  8,  1, 26,  1,  1,  1, 49,  2,  1,  1,  1,
        1,  5,  1,  1,  1,  1,  1,  1, 43, 20,  1,  1,  1,  1,  1,  1,  1,
        1, 22,  2,  1,  1,  1,  1,  1,  1,  1, 11,  1,  1,  4, 43])

In [45]:
edges2 = {}

degree = (np.random.power(0.1, 100) * 50).astype(int) + 1

for i in range(100):
    pois = list(np.random.choice(50, degree[i]))
    for j in pois:
        if j not in edges2:
            edges2[j] = {}
        edges2[j][i] = np.random.poisson(1) + 1

In [46]:
redges2 = {}

for j in edges2:
    for i in edges2[j]:
        if i not in redges2:
            redges2[i] = {}
        redges2[i][j] = edges2[j][i]

In [52]:
unvax_1 = np.array([int((i // 25) % 2 == 0) * 0.6 + 0.2 for i in range(50)])
unvax_2 = np.array([0.6 - degree[i]/50*0.5 for i in range(50)]) # np.array([int((i // 100) % 2 == 0) * 0.6 + 0.2 for i in range(10000)])


In [53]:
for lamb in [0,  0.5, 1.0]:

    redges = {}

    for i in range(50):
        redges[i] = {}
        for j in redges1[i]:
            if np.random.random() < 1 - lamb:
                redges[i][j] = redges1[i][j]
        for j in redges2[i]:
            if np.random.random() < lamb:
                redges[i][j] = redges2[i][j]

    edges = {}

    for i in redges:
        for j in redges[i]:
            if j not in edges:
                edges[j] = {}
            edges[j][i] = redges[i][j]

    from scipy.sparse import csr_matrix
    
    unvax = unvax_1 * (1 - lamb) + unvax_2 * lamb

    row = [j for j in edges for i in edges[j]]
    col = [i for j in edges for i in edges[j]]
    data = [edges[j][i] for j in edges for i in edges[j]]

    bipartite = csr_matrix((data, (row, col)), shape=(50, 50))
    prod = bipartite.T @ bipartite
    prod = np.array(prod.todense())
    
    fo = open('node_synthetic_%f.csv' % (lamb), 'w')

    fo.write('Id,Unvax,centrality\n')
    for i in range(50):
        fo.write('%d,%f,%f\n' % (i, unvax[i], np.sum(prod[i])))

    fo.close()
    
    fo = open('edge_synthetic_%f.csv' % (lamb), 'w')

    fo.write('Source,Target,Weight\n')
    for i in range(50):
        for j in range(50):
            if prod[i][j] and i != j:
                fo.write('%d,%d,%f\n' % (i, j, prod[i, j]))

    fo.close()
    

In [2]:
dict_param = np.load('../Aug-01/dict_param_synthetic_0.500000.npy', allow_pickle=True).item()

In [4]:
bipartite = dict_param['poi_cbg_visits_list'][0]

In [10]:
prod = bipartite.T @ bipartite

In [11]:
prod

<10000x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 6536150 stored elements in Compressed Sparse Column format>

In [12]:
prod = np.array(prod.todense())

In [21]:
prod = prod[:200, :200]

In [35]:
fo = open('node_synthetic.csv', 'w')

fo.write('Id,Unvax,centrality\n')
for i in range(200):
    fo.write('%d,%f,%f\n' % (i, dict_param['unvax'][i], np.sum(prod[i])))

fo.close()

In [33]:
fo = open('edge_synthetic.csv', 'w')

fo.write('Source,Target,Weight\n')
for i in range(200):
    for j in range(200):
        if prod[i][j] and i != j:
            fo.write('%d,%d,%f\n' % (i, j, prod[i, j]))

fo.close()